In [1]:
#Trying to replicate the results of "Adaptive activation functions accelerate convergence in deep and physics-informed neural networks"
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())


cuda:1


In [2]:
def true_smooth(x): #[-3,3]
    term1 = (np.power(x,3) - x)
    term2 = np.sin(7*x)/7  
    
    term3 = np.sin(12*x)
    term12 = np.multiply(term1,term2)

    return 50*(term12  + term3)

In [3]:
# Domain bounds
label = "Regr_smooth_tanh"
loss_thresh = 0.1

lb = np.array(-3.0)  # [-1. 0.]
ub = np.array(3.0) # [1.  0.99]

x_test = np.linspace(lb,ub,1000).reshape(-1,1)
u_true = true_smooth(x_test)

u_true_norm = np.linalg.norm(u_true,2)

In [4]:
def trainingdata(N_train,seed):
    np.random.seed(seed)
    
    x_train = np.random.uniform(low=lb, high=ub, size=(N_train,)).reshape(-1,1)

    u_train = true_smooth(x_train)

    return x_train, u_train

In [5]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()
     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
     
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)

            
            
    'foward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z) 
    
        a = self.linears[-1](a) 
        
        return a
    
    
    def loss(self,x,u):

        loss_val = self.loss_function(self.forward(x),u)
        
        return loss_val
        
    'test neural network'
    def test(self):
        u_pred = self.forward(x_test_tensor)       
        u_pred = u_pred.cpu().detach().numpy()
        
        return u_pred
    
    def test_loss(self):
        u_pred = self.test()
        
        test_mse_loss = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re_loss = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse_loss, test_re_loss  

In [6]:
def data_update(loss_np):
    train_loss.append(loss_np)
    test_mse, test_re = model_NN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [7]:
def train_step():
    optimizer.zero_grad()     # zeroes the gradient buffers of all parameters
    loss = model_NN.loss(x_train,u_train)
    loss.backward() #backprop
    optimizer.step()
    
    return loss

In [8]:
def train_model(max_iter,rep):
    print(rep) 
   
    torch.manual_seed(rep*123)
    start_time = time.time()
    thresh_flag = 0
    
    for i in range(max_iter):
        loss_np = train_step().cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1
                
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    
    elapsed_time[rep] = time.time() - start_time
    print('Training time: %.2f' % (elapsed_time[rep]))

In [9]:
max_reps = 5
max_iter = 3000
layers = np.array([1,50,50,50,50,1])

train_loss_full = []
test_mse_full = []
test_re_full = []

elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))


for reps in range(max_reps):
    
    train_loss = []
    test_mse_loss = []
    test_re_loss = []

    'Generate Training data'
    N_train = 300
    
    x_train_np_array, u_train_np_array = trainingdata(N_train,reps*1234)

    'Convert to tensor and send to GPU'
    x_train = torch.from_numpy(x_train_np_array).float().to(device)
    u_train = torch.from_numpy(u_train_np_array).float().to(device)
    
    x_test_tensor = torch.from_numpy(x_test).float().to(device)
       
     
    model_NN = Sequentialmodel(layers)
    model_NN.to(device)
    print(model_NN)
    params = list(model_NN.parameters())
    #optimizer = optim.Adam(PINN.parameters(), lr=0.0008,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
    optimizer = optim.Adam(model_NN.parameters(), lr=0.008)

    train_model(max_iter,reps)
 
    torch.save(model_NN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)

                 

mdic = {"train_loss": train_loss_full,"test_mse": test_mse_full, "test_re": test_re_full,"Time": elapsed_time, "Thresh_time": time_threshold, "epoch_thresh": epoch_threshold, "label": label}
savemat(label+'.mat', mdic) 

Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 3616.8254 Test MSE 3497.5138261527004 Test RE 0.9985769334061732
1 Train Loss 3596.9756 Test MSE 3490.0829045904707 Test RE 0.9975155662751474
2 Train Loss 3577.583 Test MSE 3485.7171748920614 Test RE 0.9968914770801255
3 Train Loss 3560.8613 Test MSE 3484.8863178572205 Test RE 0.9967726602602314
4 Train Loss 3548.3625 Test MSE 3486.4709462642113 Test RE 0.9969992580043905
5 Train Loss 3539.8525 Test MSE 3488.913315968127 Test RE 0.9973484096990389
6 Train Loss 3534.2415 Test MSE 3491.3956630878097 Test RE 0.9977031512059886
7 Train Loss 3530.6826 Test MSE 3493.7649831542567 Test 

100 Train Loss 3526.6433 Test MSE 3492.2443532067437 Test RE 0.9978244049161772
101 Train Loss 3526.6433 Test MSE 3492.249480824657 Test RE 0.9978251374623418
102 Train Loss 3526.6433 Test MSE 3492.260728431168 Test RE 0.9978267443257031
103 Train Loss 3526.6433 Test MSE 3492.276644332846 Test RE 0.9978290181098639
104 Train Loss 3526.6433 Test MSE 3492.295411103675 Test RE 0.9978316991694217
105 Train Loss 3526.6433 Test MSE 3492.3151572099755 Test RE 0.9978345201310862
106 Train Loss 3526.6433 Test MSE 3492.3340881607605 Test RE 0.9978372246307865
107 Train Loss 3526.643 Test MSE 3492.350628457195 Test RE 0.9978395875925665
108 Train Loss 3526.6433 Test MSE 3492.363545329715 Test RE 0.997841432904822
109 Train Loss 3526.6433 Test MSE 3492.372038890789 Test RE 0.9978426462982574
110 Train Loss 3526.6433 Test MSE 3492.37570259606 Test RE 0.9978431696961606
111 Train Loss 3526.6433 Test MSE 3492.374731613353 Test RE 0.997843030981341
112 Train Loss 3526.6433 Test MSE 3492.369575171321 T

248 Train Loss 3526.6433 Test MSE 3492.3272382721043 Test RE 0.9978362460478295
249 Train Loss 3526.6433 Test MSE 3492.3272526477735 Test RE 0.997836248101555
250 Train Loss 3526.6433 Test MSE 3492.327265980264 Test RE 0.997836250006251
251 Train Loss 3526.6433 Test MSE 3492.327266268843 Test RE 0.9978362500474776
252 Train Loss 3526.6433 Test MSE 3492.327292643036 Test RE 0.997836253815327
253 Train Loss 3526.6433 Test MSE 3492.3273105939297 Test RE 0.9978362563798133
254 Train Loss 3526.6433 Test MSE 3492.327318216184 Test RE 0.9978362574687379
255 Train Loss 3526.6433 Test MSE 3492.3272870657333 Test RE 0.9978362530185467
256 Train Loss 3526.6433 Test MSE 3492.327301295298 Test RE 0.9978362550513995
257 Train Loss 3526.6433 Test MSE 3492.327303337741 Test RE 0.9978362553431855
258 Train Loss 3526.6433 Test MSE 3492.3272899577896 Test RE 0.9978362534317092
259 Train Loss 3526.6433 Test MSE 3492.3272572748197 Test RE 0.9978362487625807
260 Train Loss 3526.6433 Test MSE 3492.3272553709

440 Train Loss 3526.5767 Test MSE 3492.2972672838205 Test RE 0.9978319643467513
441 Train Loss 3526.5605 Test MSE 3492.3068410394835 Test RE 0.9978333320700778
442 Train Loss 3526.54 Test MSE 3492.330929674192 Test RE 0.9978367734058885
443 Train Loss 3526.5134 Test MSE 3492.371167459555 Test RE 0.9978425218053206
444 Train Loss 3526.4758 Test MSE 3492.3490527999716 Test RE 0.9978393624929848
445 Train Loss 3526.4233 Test MSE 3492.2885312777016 Test RE 0.9978307163041406
446 Train Loss 3526.3467 Test MSE 3492.2731158051965 Test RE 0.9978285140163273
447 Train Loss 3526.2393 Test MSE 3491.834788709755 Test RE 0.997765891623294
448 Train Loss 3526.105 Test MSE 3493.2050718824817 Test RE 0.9979616465469447
449 Train Loss 3526.2993 Test MSE 3491.076452697237 Test RE 0.9976575412947886
450 Train Loss 3525.928 Test MSE 3491.5141373918223 Test RE 0.9977200787036326
451 Train Loss 3525.4426 Test MSE 3492.5155190354008 Test RE 0.9978631437044796
452 Train Loss 3525.4363 Test MSE 3490.4918126971

632 Train Loss 3048.768 Test MSE 3336.4886134249837 Test RE 0.9753188847711316
633 Train Loss 3114.649 Test MSE 3317.9177993528983 Test RE 0.9726007967536766
634 Train Loss 3086.7966 Test MSE 3288.5415661229704 Test RE 0.9682856098710518
635 Train Loss 3039.7878 Test MSE 3281.007101808067 Test RE 0.9671757414450218
636 Train Loss 3031.9302 Test MSE 3171.1074045654045 Test RE 0.9508396548556597
637 Train Loss 2969.9934 Test MSE 3107.9397523557504 Test RE 0.9413217760716145
638 Train Loss 2942.002 Test MSE 3119.3599945132473 Test RE 0.9430496514739523
639 Train Loss 2979.7231 Test MSE 3051.1346220938517 Test RE 0.9326796367007655
640 Train Loss 2947.0571 Test MSE 2924.9768029075963 Test RE 0.9131939444736618
641 Train Loss 2872.5117 Test MSE 2866.1265115230467 Test RE 0.9039605714858987
642 Train Loss 2797.1042 Test MSE 2929.8030346756946 Test RE 0.9139470220888969
643 Train Loss 2901.9436 Test MSE 3222.507134937884 Test RE 0.958514646945439
644 Train Loss 2987.1375 Test MSE 3213.3286455

...

954 Train Loss 1248.1383 Test MSE 1595.6683912470817 Test RE 0.6744862970875872
955 Train Loss 1247.6377 Test MSE 1596.064707527438 Test RE 0.6745700529938881
956 Train Loss 1248.2859 Test MSE 1596.8987096327005 Test RE 0.6747462737219885
957 Train Loss 1248.9998 Test MSE 1596.4588601163457 Test RE 0.6746533413212322
958 Train Loss 1249.0375 Test MSE 1596.1160133873313 Test RE 0.6745808950099845
959 Train Loss 1248.0404 Test MSE 1594.4140404056027 Test RE 0.6742211389907372
960 Train Loss 1246.5677 Test MSE 1593.3897145992369 Test RE 0.6740045286712233
961 Train Loss 1245.0938 Test MSE 1592.259407641951 Test RE 0.6737654260980606
962 Train Loss 1244.062 Test MSE 1591.7429931533275 Test RE 0.6736561567030305
963 Train Loss 1243.5244 Test MSE 1591.6723651063867 Test RE 0.6736412109653511
964 Train Loss 1243.3641 Test MSE 1591.3252343865354 Test RE 0.6735677491415695
965 Train Loss 1243.4752 Test MSE 1591.9891502330308 Test RE 0.6737082438874583
966 Train Loss 1243.8997 Test MSE 1592.2545

1063 Train Loss 1215.1703 Test MSE 1553.3602337541588 Test RE 0.6654844336541411
1064 Train Loss 1209.3075 Test MSE 1551.687419839341 Test RE 0.6651260069776255
1065 Train Loss 1207.3378 Test MSE 1555.5601500470698 Test RE 0.6659555066349088
1066 Train Loss 1211.4026 Test MSE 1555.423049969408 Test RE 0.6659261588261745
1067 Train Loss 1211.7996 Test MSE 1551.99924472189 Test RE 0.6651928350054734
1068 Train Loss 1207.6888 Test MSE 1549.5246960124514 Test RE 0.6646623229166813
1069 Train Loss 1205.158 Test MSE 1551.1718468420543 Test RE 0.6650154984130173
1070 Train Loss 1207.389 Test MSE 1553.426563360401 Test RE 0.6654986418359521
1071 Train Loss 1209.67 Test MSE 1551.8071487962532 Test RE 0.6651516672059375
1072 Train Loss 1208.2983 Test MSE 1549.511842702756 Test RE 0.6646595662234779
1073 Train Loss 1205.3074 Test MSE 1547.805476913266 Test RE 0.6642934945593862
1074 Train Loss 1203.5676 Test MSE 1548.2025969645792 Test RE 0.6643787078990802
1075 Train Loss 1204.358 Test MSE 1549.

1172 Train Loss 1191.2632 Test MSE 1527.2065630577238 Test RE 0.6598583257004578
1173 Train Loss 1185.4536 Test MSE 1522.835814141888 Test RE 0.6589134170055987
1174 Train Loss 1181.2023 Test MSE 1525.6019521213896 Test RE 0.6595115834267526
1175 Train Loss 1184.5657 Test MSE 1525.5186937468723 Test RE 0.6594935870511188
1176 Train Loss 1183.7579 Test MSE 1522.0080975319931 Test RE 0.6587343209713681
1177 Train Loss 1180.7505 Test MSE 1521.9364930833128 Test RE 0.6587188253696232
1178 Train Loss 1180.7612 Test MSE 1524.0247298857485 Test RE 0.6591705818569242
1179 Train Loss 1182.1857 Test MSE 1522.2415511187457 Test RE 0.6587848390972586
1180 Train Loss 1181.2769 Test MSE 1520.7424823794634 Test RE 0.6584603810604149
1181 Train Loss 1179.2144 Test MSE 1521.7899959576237 Test RE 0.6586871214393623
1182 Train Loss 1180.0498 Test MSE 1522.1373280905216 Test RE 0.6587622862618985
1183 Train Loss 1181.2915 Test MSE 1521.2801047245207 Test RE 0.6585767622809325
1184 Train Loss 1179.4921 Tes

1283 Train Loss 1312.8608 Test MSE 1568.027636907501 Test RE 0.6686189271343365
1284 Train Loss 1231.7664 Test MSE 1555.2065834138136 Test RE 0.6658798190996416
1285 Train Loss 1213.1598 Test MSE 1549.873705463568 Test RE 0.6647371717952917
1286 Train Loss 1201.6155 Test MSE 1609.7461782846678 Test RE 0.677455091657342
1287 Train Loss 1241.3538 Test MSE 1608.214131023806 Test RE 0.6771326370063488
1288 Train Loss 1243.7456 Test MSE 1561.1535912365232 Test RE 0.667151744665166
1289 Train Loss 1207.3451 Test MSE 1541.6344304004247 Test RE 0.6629679144695646
1290 Train Loss 1195.4438 Test MSE 1528.2077669006437 Test RE 0.6600745847401641
1291 Train Loss 1184.1454 Test MSE 1537.6730725357584 Test RE 0.6621155909954625
1292 Train Loss 1194.6144 Test MSE 1533.1108562107745 Test RE 0.6611326257188983
1293 Train Loss 1198.758 Test MSE 1531.0449043779074 Test RE 0.6606870191241785
1294 Train Loss 1201.115 Test MSE 1527.540174191004 Test RE 0.6599303932480164
1295 Train Loss 1188.5027 Test MSE 1

KeyboardInterrupt: 

In [ ]:
u_pred = model_NN.test()

In [ ]:
plt.plot(x_test,u_pred,'rx')
plt.plot(x_test,u_true,'b')

In [ ]:
a = 0
for i in range(10):
    a = a + test_re_full[i][-1]
print(a/10)